# Variational autoencoder with TFP

__Objective:__ build a variational autoencoder (VAE) using Tensorflow Probability.

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Conv2D, Flatten, Dense,
    Reshape, Conv2DTranspose)
import tensorflow_probability as tfp

tfd = tfp.distributions

## Build the model

In [ ]:
event_shape = (28, 28, 1)
latent_size = 2

### Encoder

Given a sample $x$, the encoder models the distribution of latent vectors $z$ given $x$. The true distribution $p(z | x)$ is unknown: the encoder provides an **approximation** $q(z | x)$ to that.

In [ ]:
encoder = Sequential([
    Conv2D(filters=8, kernel_size=(5, 5), strides=2, activation='tanh', input_shape=event_shape),
    Conv2D(filters=8, kernel_size=(5, 5), strides=2, activation='tanh'),
    Flatten(),
    Dense(units=64, activation='tanh'),
    # The last dense layer parametrizes the mean and (diagonal entries of)
    # the covariance matrix of a multivariate Gaussian.
    Dense(units=2 * latent_size),
    tfp.layers.DistributionLambda(
        lambda t: tfd.MultivariateNormalDiag(
            loc=t[..., :latent_size],
            scale_diag=tf.math.exp(t[..., latent_size:])
        )
    )
])

In [ ]:
encoder(tf.random.normal(shape=(13, 28, 28, 1)))

### Decoder

The decoder "decodes" the latent vectors back into the samples, so given a latent vector $z$ it models the distribution of samples, $p(x | z)$.

In [ ]:
decoder = Sequential([
    Dense(units=64, activation='tanh', input_shape=(latent_size,)),
    Dense(units=128, activation='tanh'),
    Reshape((4, 4, 8)),
    Conv2DTranspose(filters=8, kernel_size=(5, 5), strides=2, output_padding=1, activation='tanh'),
    Conv2DTranspose(filters=8, kernel_size=(5, 5), strides=2, output_padding=1, activation='tanh'),
    Conv2D(filters=1, kernel_size=(3, 3), padding='same'),
    Flatten(),
    tfp.layers.IndependentBernoulli(event_shape)
])